In [2]:
import pyodbc
import pandas as pd
import config as cfg

In [18]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
cnxn = pyodbc.connect( 'DRIVER={ODBC Driver 13 for SQL Server};SERVER=' + cfg.mssql['server'] + ';DATABASE=' 
                      + cfg.mssql['database'] + ';UID=' + cfg.mssql['username'] + ';PWD=' + cfg.mssql['password'] )

In [5]:
query = "SELECT * FROM BankView WHERE [State]='TX';"
data = pd.read_sql(query, cnxn, index_col='BankID')
data.head()

,UniqueNum,Name,Address1,Address2,City,State,Zip,Deposit,Lat,Lng,...,MeanPSDistance,PSCount,Take,PDistance,Officers1000,FFLCount,AvgRating,Target,Population,CrimeRate1000
BankID,,,,,,,,,,,,,,,,,,,,,
1,6371,"Austin Bank, Texas National Association",169 S. Frankston Hwy,,Frankston,TX,75763,89823000,32.159764,-95.455885,...,8.073403,2,44911,0.000000,0.000000,3,NaN,None,1163.0,42.13
2,221632,"Austin Bank, Texas National Association",2745 South Loop 256,,Palestine,TX,75801,74868000,31.733813,-95.623431,...,2.158156,1,37434,0.484184,1.772727,6,4.8,None,18194.0,39.13
3,16168,"Capital One, National Association",2121 South Loop 256,,Palestine,TX,75801,105194000,31.735576,-95.612655,...,2.316097,1,52597,0.468390,1.772727,6,NaN,None,18194.0,39.13
4,2196,Citizens National Bank,207 West Spring Street,,Palestine,TX,75801,92886000,31.761967,-95.633062,...,0.134378,1,46443,0.686562,1.772727,6,2.7,None,18194.0,39.13
5,569848,"Commercial Bank of Texas, National Association",109 West Parker St,,Elkhart,TX,75839,33338000,31.625053,-95.579769,...,10.500000,0,16669,0.000000,0.000000,1,5.0,None,NaN,NaN


In [6]:
data['CrimeRate1000'].mean()

36.84859592711564

In [7]:
data['Population'].mean()

598919.9247945695

In [8]:
data.isnull().sum()

UniqueNum               0
Name                    0
Address1                0
Address2                0
City                    0
State                   0
Zip                     0
Deposit                 0
Lat                     8
Lng                     8
ClosestStationID      419
ClosestPSDistance       0
MeanPSDistance          0
PSCount                 0
Take                    0
PDistance               0
Officers1000            0
FFLCount                0
AvgRating            2404
Target               6507
Population            909
CrimeRate1000         909
dtype: int64

In [9]:
values = {'CrimeRate1000': data['CrimeRate1000'].mean(), 'Population': data['Population'].mean(), 'AvgRating' : data['AvgRating'].mean()}
data.fillna(value=values, inplace=True)
data.shape

(6507, 22)

## Use Model to Predict banks to Target

In [10]:
try:
    import cPickle as pickle
except ImportError:
    import pickle
    
## Load Model
model_filepath = 'targetbanks_randomforestclassifier.pkl'
in_logreg = open(model_filepath, 'rb')
rfc = pickle.load(in_logreg)
in_logreg.close()

In [11]:
feature_cols = ['ClosestPSDistance', 'Take', 'PDistance', 'Officers1000',
       'FFLCount', 'AvgRating', 'Population', 'CrimeRate1000']
X = data[feature_cols]
data['y_pred'] = rfc.predict(X)

In [12]:
# Total TARGETABLE banks
tgtsum = data[data.y_pred == 1].Name.count();
total = data.Name.count()
print(tgtsum, 'out of', total, ' pct:', round((tgtsum/total * 100)), '%')

1061 out of 6507  pct: 16.0 %


## Show Result

In [13]:
# Drop columns
dftarget = data[data.y_pred == 1].copy()
dftarget.drop(['UniqueNum', 
       'Deposit', 'Lat', 'Lng', 'ClosestStationID',
       'MeanPSDistance', 'PSCount', 'Target', 'Population'], axis=1, inplace=True)

## Top 5 banks with highest Take

In [31]:
# Top 5 banks with highest Take
dftop5 = dftarget.sort_values(by=['Take'], ascending=[False]).head(5)
dftop5

,Name,Address1,Address2,City,State,Zip,ClosestPSDistance,Take,PDistance,Officers1000,FFLCount,AvgRating,CrimeRate1000,y_pred
BankID,,,,,,,,,,,,,,
429,USAA Federal Savings Bank,10750 Mcdermott Freeway,,San Antonio,TX,78288,3.287828,36000101,0.371217,4.054054,29,3.000000,59.25,1
1037,LegacyTexas Bank,5851 Legacy Circle,,Plano,TX,75024,3.452618,1638172,0.354738,1.396770,33,3.000000,20.44,1
601,"Wells Fargo Bank, National Association",9821 Broadway,,Pearland,TX,77584,4.944473,1099152,0.205553,2.318182,16,3.708603,19.93,1
3379,"Texas Capital Bank, National Association","1 Riverway, Suite 150",,Houston,TX,77056,3.244498,1097901,0.375550,2.512702,76,1.000000,53.76,1
1743,"TIB The Independent Bankersbank, National Asso...",11701 Luna Road,,Dallas,TX,75234,4.074525,742480,0.292548,2.840000,27,1.800000,41.90,1


# Use K-Means with RandomForest

In [17]:
feature_cols = [ 'Take', 'PDistance', 'Officers1000', 'FFLCount', 'AvgRating', 'CrimeRate1000']
X = data[feature_cols]

In [36]:
# K-means with 2 clusters is overfit and NOT the best for this data, so try 4,5,6?
km = KMeans(n_clusters=5, random_state=1)
km.fit(X)
data['cluster'] = km.labels_

In [37]:
data_X = X.copy()
data_X['cluster'] = km.labels_
centers = data_X.groupby('cluster').mean()
centers

,Take,PDistance,Officers1000,FFLCount,AvgRating,CrimeRate1000
cluster,,,,,,
0,3.867319e+04,0.425218,1.840943,26.063561,3.710063,36.813444
1,3.734823e+07,0.520455,3.441114,39.000000,3.500000,56.505000
2,1.338549e+07,0.600560,2.756409,35.000000,3.708603,41.900000
3,1.991908e+06,0.518433,2.347750,43.285714,3.330620,45.345714
4,2.848512e+07,0.600889,2.756409,35.000000,2.600000,41.900000


In [38]:
data[data.y_pred == 1].groupby('cluster').mean()

,Deposit,Lat,Lng,ClosestStationID,ClosestPSDistance,MeanPSDistance,PSCount,Take,PDistance,Officers1000,FFLCount,AvgRating,Population,CrimeRate1000,y_pred
cluster,,,,,,,,,,,,,,,
0,1.367252e+08,30.937460,-97.328611,351.151648,5.513488,6.701740,2.921476,6.836234e+04,0.215466,1.060054,21.938505,3.593837,7.241017e+05,38.845382,1.0
1,7.200020e+10,29.541059,-98.575308,580.000000,3.287828,5.779003,8.000000,3.600010e+07,0.371217,4.054054,29.000000,3.000000,1.498642e+06,59.250000,1.0
3,2.556818e+09,30.799651,-95.886048,129.000000,3.880530,5.531378,4.666667,1.278408e+06,0.311947,2.075884,41.666667,2.569534,9.118013e+05,31.376667,1.0


In [24]:
# Drop columns
dftarget2 = data[data.y_pred == 1].copy()
dftarget2.drop(['UniqueNum', 
       'Deposit', 'Lat', 'Lng', 'ClosestStationID',
       'MeanPSDistance', 'PSCount', 'Target', 'Population'], axis=1, inplace=True)

## Top 5 banks with highest Take

In [39]:
# Top 5 banks with highest Take and lower pDistance values from K-Means centers
dftop5 = dftarget2[dftarget2.cluster.isin([0, 3])].sort_values(by=['Take'], ascending=[False]).head(5)
dftop5

,Name,Address1,Address2,City,State,Zip,ClosestPSDistance,Take,PDistance,Officers1000,FFLCount,AvgRating,CrimeRate1000,y_pred,cluster
BankID,,,,,,,,,,,,,,,
1037,LegacyTexas Bank,5851 Legacy Circle,,Plano,TX,75024,3.452618,1638172,0.354738,1.396770,33,3.000000,20.44,1,3
601,"Wells Fargo Bank, National Association",9821 Broadway,,Pearland,TX,77584,4.944473,1099152,0.205553,2.318182,16,3.708603,19.93,1,3
3379,"Texas Capital Bank, National Association","1 Riverway, Suite 150",,Houston,TX,77056,3.244498,1097901,0.375550,2.512702,76,1.000000,53.76,1,3
1743,"TIB The Independent Bankersbank, National Asso...",11701 Luna Road,,Dallas,TX,75234,4.074525,742480,0.292548,2.840000,27,1.800000,41.90,1,0
6189,Compass Bank,700 San Bernardo Avenue,,Laredo,TX,78040,3.121091,713798,0.387891,2.000000,29,4.200000,34.25,1,0


Not in this list: 
- 429 USAA Federal Savings Bank 

Not in earlier list: 
- 6189 Compass Bank


## Top 5 of banks with longest distance to Police Station

In [40]:
# Top 5 of banks with highest distance to Police Station
dftop5 = dftarget2[dftarget2.cluster.isin([0, 3])].sort_values(by=['ClosestPSDistance'], ascending=False).head(5)
dftop5

,Name,Address1,Address2,City,State,Zip,ClosestPSDistance,Take,PDistance,Officers1000,FFLCount,AvgRating,CrimeRate1000,y_pred,cluster
BankID,,,,,,,,,,,,,,,
6505,Zapata National Bank,703 Hidalgo Blvd,,Zapata,TX,78076,10.5,37399,0.0,0.0,4,4.500000,36.848596,1,0
860,City Bank,107 West Taylor Street,,Morton,TX,79346,10.5,29222,0.0,0.0,1,3.708603,36.848596,1,0
3412,Trustmark National Bank,6809 Fm 1960 West,,Houston,TX,77069,10.5,47442,0.0,0.0,42,3.708603,53.760000,1,0
3357,Regions Bank,12100 Huffmeister,,Cypress,TX,77429,10.5,17599,0.0,0.0,34,3.900000,36.848596,1,0
3291,"MERCANTIL BANK, NATIONAL ASSOCIATION",12145 Fm 1960 Road West,,Houston,TX,77065,10.5,42481,0.0,0.0,30,3.708603,53.760000,1,0


## Sort by lowest officers rate per 1000

In [41]:
#Sort by lowest officers rate per 1000
dftop5 = dftarget2[dftarget2.cluster.isin([0, 3])].sort_values(by=['Officers1000'], ascending=True).head(5)
dftop5

,Name,Address1,Address2,City,State,Zip,ClosestPSDistance,Take,PDistance,Officers1000,FFLCount,AvgRating,CrimeRate1000,y_pred,cluster
BankID,,,,,,,,,,,,,,,
1,"Austin Bank, Texas National Association",169 S. Frankston Hwy,,Frankston,TX,75763,8.008239,44911,0.000000,0.0,3,3.708603,42.130000,1,0
3425,"Wells Fargo Bank, National Association",811 Sheldon Rd,,Channelview,TX,77530,6.130952,55341,0.086905,0.0,15,3.700000,36.848596,1,0
3427,"Wells Fargo Bank, National Association",17802 Cypress Road,,Cypress,TX,77429,8.499471,34156,0.000000,0.0,18,3.708603,36.848596,1,0
3428,"Wells Fargo Bank, National Association",13150 Louetta Road,,Cypress,TX,77429,6.935804,66998,0.006420,0.0,30,3.700000,36.848596,1,0
3429,"Wells Fargo Bank, National Association",25694 Highway 290,,Cypress,TX,77429,9.778729,53090,0.000000,0.0,17,2.800000,36.848596,1,0
